<h1>Importações</h1>


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import MetaTrader5 as mt5
import datetime
import numpy as np
data = pd.DataFrame(pd.read_excel('graf_diario2.xlsx'))
data = data[::-1]
data = data.reset_index()
del data['index']
data['Amplitude_Vwap'] = abs(data['VWAP D'] - data['VWAP D'].shift(1))
data['Amplitude'] = data['Máxima'] - data['Mínima']
data['media_amplitude'] = data['Amplitude'].rolling(window=22,closed='left').mean()
data['teste'] = data['VWAP D'].shift(1) 
data

,Data,Abertura,Máxima,Mínima,Fechamento,VWAP D,VWAP S,Prior Cote,Prior Cote.1,Prior Cote.2,Prior Cote.3,Prior Cote.4,VWAP M,Prior Cote Ajuste,Média Móvel A [200],Volume Profile Diário,Amplitude_Vwap,Amplitude,media_amplitude,teste
0,2005-05-17,24500,24750,24200,24670,24530.00,36705.65,NaN,NaN,NaN,NaN,NaN,36705.65,NaN,NaN,NaN,NaN,550,NaN,NaN
1,2005-05-18,24650,25150,24650,25050,24875.00,37187.42,NaN,NaN,NaN,NaN,NaN,37187.42,NaN,NaN,NaN,345.00,500,NaN,24530.00
2,2005-05-19,24750,25050,24740,25050,24897.50,37204.81,NaN,NaN,NaN,NaN,NaN,37204.81,NaN,NaN,NaN,22.50,310,NaN,24875.00
3,2005-05-20,25000,25000,24680,24750,24857.50,37203.36,NaN,NaN,NaN,NaN,NaN,37203.36,NaN,NaN,NaN,40.00,320,NaN,24897.50
4,2005-05-23,24850,24850,24350,24350,24600.00,36797.83,24750.0,0.0,24200.0,25150.0,25150.0,37145.70,0.0,NaN,NaN,257.50,500,NaN,24857.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4598,2023-12-12,127410,127590,126010,126525,126618.70,126779.25,127240.0,127287.0,125670.0,128195.0,128195.0,126999.87,127287.0,115089.38,NaN,303.03,1580,1446.363636,126921.73
4599,2023-12-13,128690,132325,128315,132210,129533.26,127802.01,127240.0,127287.0,125670.0,128195.0,128195.0,127295.96,127287.0,115212.98,NaN,2914.56,4010,1430.000000,126618.70
4600,2023-12-14,132625,133350,132405,132910,132869.03,129064.08,127240.0,127287.0,125670.0,128195.0,128195.0,127822.74,127287.0,115342.90,NaN,3335.77,945,1535.000000,129533.26
4601,2023-12-15,133100,133760,131930,132320,132704.00,129736.09,127240.0,127287.0,125670.0,128195.0,128195.0,128208.98,127287.0,115475.78,NaN,165.03,1830,1541.818182,132869.03


<h1>Probabilidade de vir buscar a VWAP D do dia anterior</h1>

In [11]:
#Levantar estudo da quantidade de vezes que ele vem buscar a vwap do dia anterior e máxima e mínima do 10m.
periodo = 63
data_1 = data[-periodo:].loc[(data['Mínima']<data['VWAP D'].shift(1)) & (data['Máxima']>data['VWAP D'].shift(1))]
data_1.shape[0]/periodo

0.746031746031746

In [12]:
#Probabilidade de ele vir buscar a vwap do dia anterior no últimos 63 dias
probabilidade_756 = 518/756
probabilidade_252 = 173/252
probabilidade_63 = 42/63
probabilidade_21 = 14/21
print(probabilidade_21,probabilidade_63,probabilidade_252,probabilidade_756, sep='*-*-*-*-*')
#Conclusão temos a probabilidade de 66% de vir buscar a vwap do dia anterior

0.6666666666666666*-*-*-*-*0.6666666666666666*-*-*-*-*0.6865079365079365*-*-*-*-*0.6851851851851852


<h1>Probabildiade de ele vir buscar a VWAP D-1, após ter tido um dia com amplitude acima do terceiro quartil</h1>

In [13]:
data_1 = data[-periodo:].loc[(data['Mínima']<data['VWAP D'].shift(1)) & (data['Máxima']>data['VWAP D'].shift(1)) & (data['Amplitude'].shift(1)>data['Amplitude'].quantile(q=0.75))]

In [14]:
total_dias,_ = data[-periodo:].loc[data['Amplitude'].shift(1)>data['Amplitude'].quantile(q=0.75)].shape
ocorreu,_=data_1.shape
print(ocorreu/total_dias,total_dias)

0.55 20


<h1>Probabildiade de ele vir buscar a VWAP D-1, após ter tido um dia com amplitude ABAIXO da MEDIANA</h1>

In [15]:
data_2 = data[-periodo:].loc[(data['Mínima']<data['VWAP D'].shift(1)) & (data['Máxima']>data['VWAP D'].shift(1)) & (data['Amplitude'].shift(1)<data['Amplitude'].quantile(q=0.5))]
total_dias_2,_ = data[-periodo:].loc[data['Amplitude'].shift(1)<data['Amplitude'].quantile(q=0.5)].shape
ocorreu_2,_=data_2.shape
print(ocorreu_2/total_dias_2,total_dias_2)

0.9411764705882353 17


<h1>Probabildiade de ele vir buscar a VWAP D-1, após ter tido um dia com amplitude ABAIXO da MÉDIA</h1>

In [16]:
data_3 = data[-periodo:].loc[(data['Mínima']<data['VWAP D'].shift(1)) & (data['Máxima']>data['VWAP D'].shift(1)) & (data['Amplitude'].shift(1)<data['media_amplitude'].shift(1))]
total_dias_3,_ = data[-periodo:].loc[data['Amplitude'].shift(1)<data['media_amplitude'].shift(1)].shape
ocorreu_3,_=data_3.shape
print(ocorreu_3/total_dias_3)

0.8717948717948718


<h1>Probabildiade de ele vir buscar a VWAP D-1, após ter tido um dia com amplitude ACIMA da MÉDIA</h1>

In [8]:
data_4 = data[-periodo:].loc[(data['Mínima']<data['VWAP D'].shift(1)) & (data['Máxima']>data['VWAP D'].shift(1)) & (data['Amplitude'].shift(1)>data['media_amplitude'].shift(1))]
total_dias_4,_ = data[-periodo:].loc[data['Amplitude'].shift(1)>data['media_amplitude'].shift(1)].shape
ocorreu_4,_=data_4.shape
print(ocorreu_4/total_dias_4)

0.6


In [9]:
import MetaTrader5 as mt5
import datetime
import pandas as pd 
import numpy as np
mt5.initialize()
ativo ='WIN$N'
ticks = pd.DataFrame(mt5.copy_ticks_range('WIN$N',datetime.datetime(2023,12,21),datetime.datetime(2023,12,22),mt5.COPY_TICKS_ALL))
ticks['time'] =pd.to_datetime(ticks['time'],unit='s')

ticks

,time,bid,ask,last,volume,time_msc,flags,volume_real
0,2023-12-21 09:00:34,0.0,0.0,133375.0,1733,1703149234765,312,1733.0
1,2023-12-21 09:00:34,0.0,0.0,133375.0,2052,1703149234765,312,2052.0
2,2023-12-21 09:00:34,0.0,0.0,133375.0,418,1703149234766,312,418.0
3,2023-12-21 09:00:34,0.0,0.0,133375.0,92,1703149234767,312,92.0
4,2023-12-21 09:00:34,0.0,0.0,133375.0,551,1703149234767,312,551.0
...,...,...,...,...,...,...,...,...
1099214,2023-12-21 18:31:23,0.0,0.0,134240.0,1550,1703183483341,56,1550.0
1099215,2023-12-21 18:31:23,0.0,0.0,134240.0,919,1703183483341,56,919.0
1099216,2023-12-21 18:31:23,0.0,0.0,134240.0,607,1703183483342,56,607.0
1099217,2023-12-21 18:31:23,0.0,0.0,134240.0,291,1703183483342,56,291.0


In [10]:
ponderada = 0
for i in ticks.itertuples():
    ponderada += i.volume * i.last

print(round(ponderada/ticks['volume'].sum(),0))

133724.0
